# AOC 2020

## Day 21
### Part 1

In [ ]:
with open('input21.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
from collections import defaultdict, Counter

count = Counter()
mapping = defaultdict(list)

for line in lines:
    ingr, allerg = line.split(' (contains ')
    ingredients = ingr.split()
    count += Counter(ingredients)
    allergens = [al.strip() for al in allerg[:-1].split(",")]
    
    all_ingredients.update(set(ingredients))
    
    for al in allergens:
        mapping[al].append(set(ingredients))
    

In [ ]:
from functools import reduce

bad_ingredients = set()
for maps in mapping.values():
    bad_ingredients |= reduce(lambda a, b : a&b , maps)

In [ ]:
sum(c for key, c in count.items() if key not in bad_ingredients)

### Part 2

In [ ]:
from functools import reduce

bad_map = dict()
for key, maps in mapping.items():
    bad_map[key] = reduce(lambda a, b : a&b , maps)

In [ ]:
canonical = dict()

while len(canonical) < len(bad_map):
    for al, ing in bad_map.items():
        remaining = ing - set(canonical.keys())
        if len(remaining) == 1:
            canonical[list(remaining)[0]] = al

In [ ]:
",".join([ing for ing, al in sorted(canonical.items(), key=lambda t: t[1])])

## Day 20
### Part 1

In [ ]:
with open('input20.txt') as fin:
    lines = [line for line in fin.readlines()]
    raw_tiles = ''.join(lines).split('\n\n')[:-1]

In [ ]:
class Tile:
    def __init__(self, raw):
        lines = raw.split('\n')
        self.id = lines[0].split()[1][:-1]
        self.top = lines[1]
        self.bottom = lines[-1]
        self.left = "".join([l[0] for l in lines[1:]])
        self.right = "".join([l[-1] for l in lines[1:]])
        
    def __str__(self):
        return f"{self.top} - {self.right} - {self.bottom} - {self.left}"
    
    def borders(self, flipped=False):
        if flipped:
            return set(getattr(self, attr)[::-1] for attr in ["top", "bottom", "left", "right"])
        else:
            return set(getattr(self, attr) for attr in ["top", "bottom", "left", "right"])
    
    def __repr__(self):
        return repr(self.id)

tiles = [Tile(tile) for tile in raw_tiles]

In [ ]:
def find_possible_matches(tile, tiles):
    borders = tile.borders()
    possible_matches = 0
    for other in tiles:
        if other != tile:
            if borders & (other.borders() | other.borders(flipped=True)):
                possible_matches += 1
    return possible_matches

matches = {}
for tile in tiles:
    matches[tile.id] = find_possible_matches(tile, tiles)

In [ ]:
from functools import reduce
reduce(lambda a,b : a*b, [int(k) for k, v in matches.items() if v == 2])

In [ ]:
[int(k) for k, v in matches.items() if v == 2]

### Part 2

In [ ]:
def vflip(tile):
    return tile[::-1]

def hflip(tile):
    return [t[::-1] for t in tile]

def rotate90(tile):
    rotate =[]
    for idx in range(len(tile)):
        rotate.append("".join([t[idx] for t in tile]))
    return hflip(rotate)

def possible_tiles(tile):
    tiles = []
    tile_ = tile.copy()
    tiles.extend([tile_.copy(), hflip(tile_), vflip(tile_)])
    tile_ = rotate90(tile)
    tiles.extend([tile_.copy(), hflip(tile_), vflip(tile_)])
    tiles.append(rotate90(tile_))
    tiles.append(rotate90(rotate90(tile_)))
    return tiles

class FullTile:
    def __init__(self, raw):
        lines = raw.split('\n')
        self.id = lines[0].split()[1][:-1]
        self._tiles = possible_tiles(lines[1:])
        self._ptr = 0
        
    @property
    def tile(self):
        return self._tiles[self._ptr]
    
    @property
    def subtile(self):
        return [t[1:-1] for t in self.tile[1:-1] ]
        
    @property
    def top(self):
        return self.tile[0]
    
    @property
    def bottom(self):
        return self.tile[-1]
    
    @property
    def left(self):
        return "".join([t[0] for t in self.tile ])
    
    @property
    def right(self):
        return "".join([t[-1] for t in self.tile ])
    
    def find_variant(self, target, attr):
        bckp_ptr = self._ptr
        for ptr in range(len(self._tiles)):
            self._ptr = ptr
            if getattr(self, attr) == target:
                return True
        self._ptr = bckp_ptr
        return False

In [ ]:
full_tiles = [FullTile(raw) for raw in raw_tiles]
mapping = {tile.id: tile for tile in full_tiles}

In [ ]:
tile = mapping['1723']

assigned = {tile}
to_process = [tile]

matching = defaultdict(dict)

while to_process:
    tile = to_process.pop()
    
    if tile in processed:
        continue
    
    for other in full_tiles:
        if other == tile:
            continue

        for attr_a, attr_b in zip(["top", "bottom", "left", "right"], ["bottom", "top", "right", "left"]):
            res = other.find_variant(getattr(tile, attr_a), attr_b)
            if res:
                assigned.add(other)
                to_process.append(other)
                matching[tile.id][attr_a] = other.id
                
    processed.add(tile)
        

In [ ]:
Counter([len(match) for match in matching.values()])

In [ ]:
x = 0
y = 11
map_ = defaultdict(dict)


current = mapping['1723']
current.x = x
current.y = y
to_process.append(current)

processed = set()

while to_process:
    current = to_process.pop()
    if current in processed:
        continue
    
    for link, nid in matching[current.id].items():
        other = mapping[nid]
        if link == 'left':
            other.x = current.x - 1
            other.y = current.y
        if link == 'top':
            other.x = current.x
            other.y = current.y - 1
        if link == 'right':
            other.x = current.x + 1
            other.y = current.y
        if link == 'bottom':
            other.x = current.x
            other.y = current.y + 1
        to_process.append(other)
        processed.add(current)

In [ ]:
map_ = defaultdict(dict)

for tile in full_tiles:
    map_[tile.x][tile.y] = tile

In [ ]:
fulltile = []
for j in range(12):
    fulltile.extend(["".join([map_[i][j].subtile[y] for i in range(12)]) for y in range(8)])

In [ ]:
pattern = [
    "                  # ",
    "#    ##    ##    ###",
    " #  #  #  #  #  #   "]
pattern = [p.replace(" ", ".") for p in pattern]

In [ ]:
import re

def find_dragons(fulltile):
    dragons = []
    for l1, l2, l3 in zip(fulltile[:-2], fulltile[1:-1], fulltile[2:]):
        for i in range(len(l1) - 20):
            if re.match(pattern[0], l1[i:i+20]) and re.match(pattern[1], l2[i:i+20]) and re.match(pattern[2], l3[i:i+20]):
                dragons.append([l1[i:i+20], l2[i:i+20], l3[i:i+20]])
    return dragons

In [ ]:
for idx, ftile in enumerate(possible_tiles(fulltile)):
    dragons = find_dragons(ftile)
    if dragons and False:
        for dragon in dragons:
            for l in dragon:
                print(l)
            print()
        break

In [ ]:
nb_hash_per_dragon = reduce(lambda a, b: a + b, [Counter(p) for p in pattern])['#']
nb_hash_total = reduce(lambda a, b: a + b, [Counter(p) for p in fulltile])['#']
print(nb_hash_total - (len(dragons)*nb_hash_per_dragon))

## Day 19
### Part 1

In [ ]:
with open('input19.txt') as fin:
    lines = [line for line in fin.readlines()]
    rules, messages = ''.join(lines).split('\n\n')
    rules = rules.split('\n')
    messages = messages.split('\n')

In [ ]:
rules_dict = {}
for rule in rules:
    idx, ruleset = rule.split(': ')
    ruleset = ruleset.replace("\"", "")
    rules_dict[idx] = ruleset.split(' | ')

In [ ]:
def get_valid_messages(line):
    lines = [line]
    valid_messages = set()

    while lines:
        line = lines.pop()

        if all(char not in rules_dict for char in line.split()):
            valid_messages.add(line)
            continue

        new_line = []
        second_line = None
        for cnt, idx in enumerate(line.split()):
            if idx not in rules_dict:
                new_line.append(idx)
            else:
                subrule = rules_dict[idx]
                if len(subrule) == 1:
                    new_line.append(subrule[0])
                elif len(subrule) == 2:
                    second_line = new_line.copy()
                    new_line.append(subrule[0])
                    second_line.append(subrule[1])
                break
        new_line.extend(line.split()[cnt+1:])
        lines.append(" ".join(new_line))
        if second_line:
            second_line.extend(line.split()[cnt+1:])
            lines.append(" ".join(second_line))
        
    return valid_messages

In [ ]:
valid_messages = get_valid_messages(rules_dict['0'][0])
valid_messages = set(msg.replace(" ", "") for msg in valid_messages)
len([msg for msg in messages if msg in valid_messages])

### Part 2

In [ ]:
part_a = set(msg.replace(" ", "") for msg in get_valid_messages('42'))
part_b = set(msg.replace(" ", "") for msg in get_valid_messages('31'))

In [ ]:
def translate(msg):
    new_msg = ""
    for i in range(0, len(msg), 8):
        if msg[i:i+8] in part_a:
            new_msg = f"{new_msg}A"
        elif msg[i:i+8] in part_b:
            new_msg = f"{new_msg}B"
        else:
            print("ERROR")
    return new_msg

import re

def valid_translation(msg):
    res = re.match(r"^(A+)(B+)$", msg)
    if res and len(res.group(1)) > len(res.group(2)):
        return True
    return False
    
#[(translate(msg), valid_translation(translate(msg))) for msg in messages]
len([msg for msg in messages if valid_translation(translate(msg))])

## Day 18
### Part 1

In [ ]:
with open('input18.txt') as fin:
    lines = [line.strip().replace(" ", "") for line in fin.readlines()]

In [ ]:
def add(a, b):
    return int(a) + int(b)

def mult(a, b):
    return int(a)*int(b)

def parse(line):
    val = 0
    op = add
    idx = 0

    while idx < len(line):
        elem = line[idx]
        if elem == '(':
            other, length = parse(line[idx+1:])
            val = op(val, other)
            idx += length
        elif elem == ')':
            return val, idx+1
        elif elem == '+':
            op = add
        elif elem == '*':
            op = mult
        else:
            val = op(val, elem)
        idx += 1
    return val

In [ ]:
sum(parse(line) for line in lines)

### Part 2

In [ ]:
from functools import reduce
import re

def reduce_mult(ops):
    if len(ops) > 1:
        return reduce(lambda a, b: a*b, ops)
    return int(ops[0])

def reduce_add(ops):
    if len(ops) > 1:
        return reduce(lambda a, b: int(a)+int(b), ops)
    return int(ops[0])

def parse_unary(line):
    return reduce_mult([reduce_add(elem.split('+')) for elem in line.split('*')])

def parse(line):
    res = re.match(r"(.*)(\([^\)]+\))(.*)", line)
    if res:
        return parse(f"{res.group(1)}{parse_unary(res.group(2)[1:-1])}{res.group(3)}")
    return parse_unary(line)

In [ ]:
sum(parse(line) for line in lines)

## Day 17
### Part 1

In [ ]:
with open('input17.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
active = set()

for y, line in enumerate(lines):
    for x, val in enumerate(line):
        if val == '#':
            active.add((x, y, 0))

In [ ]:
def neighbors(pt):
    nn = set()
    for x in range(pt[0]-1, pt[0]+2):
        for y in range(pt[1]-1, pt[1]+2):
            for z in range(pt[2]-1, pt[2]+2):
                nn.add((x, y, z))
    return nn - {pt}

for i in range(6):
    neighborhood = set.union(*[neighbors(pt) for pt in active])
    
    new_active = set()

    for pt in active:
        if 2 <= len(neighbors(pt) & active) <= 3:
            new_active.add(pt)

    for pt in neighborhood:
        conv = neighbors(pt) & active
        if pt in active and 2 <= len(conv) <= 3:
            new_active.add(pt)
        if pt not in active and len(conv) == 3:
            new_active.add(pt)
            
    active = new_active
    
len(new_active)

### Part 2

In [ ]:
def neighbors(pt):
    nn = set()
    for x in range(pt[0]-1, pt[0]+2):
        for y in range(pt[1]-1, pt[1]+2):
            for z in range(pt[2]-1, pt[2]+2):
                for w in range(pt[3]-1, pt[3]+2):
                    nn.add((x, y, z, w))
    return nn - {pt}

In [ ]:
active = set()

for y, line in enumerate(lines):
    for x, val in enumerate(line):
        if val == '#':
            active.add((x, y, 0, 0))

In [ ]:
for i in range(6):
    neighborhood = set.union(*[neighbors(pt) for pt in active])
    
    new_active = set()

    for pt in active:
        if 2 <= len(neighbors(pt) & active) <= 3:
            new_active.add(pt)

    for pt in neighborhood:
        conv = neighbors(pt) & active
        if pt in active and 2 <= len(conv) <= 3:
            new_active.add(pt)
        if pt not in active and len(conv) == 3:
            new_active.add(pt)
            
    active = new_active
    
len(new_active)

## Day 16
### Part 1

In [ ]:
with open('input16.txt') as fin:
    lines = [line for line in fin.readlines()]
    groups = ''.join(lines).split('\n\n')
    constraints, ticket, nearby = groups

In [ ]:
import re

def compute_constraints(constraints):

    const_dict = {}

    for const in constraints.split('\n'):
        match = re.match(r"(.*): (\d+)-(\d+) or (\d+)-(\d+)", const)
        const_dict[match.group(1)] = [(int(match.group(2)), int(match.group(3))), (int(match.group(4)), int(match.group(5)))]
        
    return const_dict

def compute_ranges(const_dict):
    ranges = []
    for const_ranges in const_dict.values():
        valid_ranges = set()
        for r_ in const_ranges:
            valid_ranges |= set(range(r_[0], r_[1]+1))
        ranges.append(valid_ranges)
    return ranges

def compute_tickets(ticket, nearby):
    my_ticket = [int(val) for val in ticket.split('\n')[-1].split(',')]
    nearby_tickets = [[int(val) for val in near_ticket.split(',')] for near_ticket in nearby.strip().split('\n')[1:]]
    return my_ticket, nearby_tickets

In [ ]:
from functools import reduce

const_dict = compute_constraints(constraints)
my_ticket, nearby_tickets = compute_tickets(ticket, nearby)

ranges = compute_ranges(const_dict)
full_ranges = reduce(lambda a, b: a|b, ranges)

invalid_values = []

for tick in nearby_tickets:
    for val in tick:
        if val not in full_ranges:
            invalid_values.append(val)
sum(invalid_values)

### Part 2

In [ ]:
valid_tickets = [tick for tick in nearby_tickets if all(v in full_ranges for v in tick)]
valid_tickets.append(my_ticket)

In [ ]:
import numpy as np

from collections import Counter

def unique_match(mapping):
    count = Counter()

    for map_ in mapping:
        count += Counter(map_)
    field, c = count.most_common()[-1]
    if c == 1:
        return field
    
def map_match(options, field):
    for idx, option in enumerate(options):
        if match in option:
            return idx

def compute_options(valid_tickets, ranges):
    options = []
    for field in np.array(valid_tickets).T:
        options.append(set(idx for idx, range_ in enumerate(ranges) if set(field) <= range_ ))
        
    return options

In [ ]:
ranges = compute_ranges(const_dict)
options = compute_options(valid_tickets, ranges)

mapping = {}
for i in range(len(options)):
    match = unique_match(options)
    idx = map_match(options, match)
    mapping[match] = idx
    options[idx] = set()
    
val = 1
for i in range(6):
    val *= my_ticket[mapping[i]]
print(val)

## Day 15

In [ ]:
inp = [0,13,1,16,6,17]

In [ ]:
from collections import defaultdict, deque

def run(inp, index):

    memory = defaultdict(deque)

    for idx, v in enumerate(inp):
        memory[v].appendleft(idx)

    last_nb = inp[-1]
    count = len(inp)

    while count < index:
        if len(memory[last_nb]) > 1:
            last_nb = memory[last_nb][0] - memory[last_nb][1]
            memory[last_nb].appendleft(count)
        else:
            last_nb = 0
            memory[0].appendleft(count)
        count += 1
    return last_nb

### Part 1

In [ ]:
run(inp, 2020)

### Part 2

In [ ]:
run(inp, 30000000)

## Day 14
### Part 1

In [ ]:
with open('input14.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
def convert(nb, mask):
    b = int(bin(int(nb))[2:])
    nb = list(f"{b:036}")
    for idx, m in enumerate(mask):
        if m != 'X':
            nb[idx] = m
    nb = "".join(nb)
    return int(f"0b{nb}", 2)

In [ ]:
memory = {}

mask = None

for line in lines:
    args = line.split()
    if args[0][:4] == 'mask':
        mask = args[-1]
    else:
        addr = int(args[0][4:-1])
        nb = args[-1]
        memory[addr] = convert(nb, mask)

sum(memory.values())

### Part 2

In [ ]:
def convert_addr(addr, mask):
    b = int(bin(int(addr))[2:])
    nb = list(f"{b:036}")
    mask_list = list(mask)
    for idx, m in enumerate(mask):
        if m != '0':
            nb[idx] = m
    return [int(f"0b{ad}", 2) for ad in  compute_addresses("".join(nb))]

def compute_addresses(mask):
    masks = []
    mask_list = list(mask)
    for idx, m in enumerate(mask):
        if m == 'X':
            for bit in ['0', '1']:
                mask_list[idx] = bit
                masks.extend(compute_masks("".join(mask_list)))
            break
    else:
        masks.append("".join(mask_list))
    return masks

In [ ]:
memory = {}

mask = None

for line in lines:
    args = line.split()
    if args[0][:4] == 'mask':
        mask = args[-1]
    else:
        addr = int(args[0][4:-1])
        nb = args[-1]
        for ad in convert_addr(addr, mask):
            memory[ad] = int(nb)

sum(memory.values())

## Day 13
### Part 1

In [ ]:
with open('input13.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
ts = int(lines[0])
buses = [int(b) for b in lines[1].split(',') if b != 'x']

In [ ]:
wait = [(bid - ts % bid) % bid for bid in buses]
best_pair = min(zip(wait, buses))
best_pair[0]*best_pair[1]

### Part 2

In [ ]:
buses = [(int(bid), idx) for idx, bid in enumerate(lines[1].split(',')) if bid != 'x']
buses = [(int(bid), (int(bid) - idx) % int(bid)) for idx, bid in enumerate(lines[1].split(',')) if bid != 'x']

In [ ]:
lines = [[], "7,13,x,x,59,x,31,19"]

In [ ]:
from functools import reduce

def chinese_remainder(buses):
    sum_ = 0
    prod = reduce(lambda a, b: a*b, [b[0] for b in buses])
    for n_i, a_i in buses:
        p = prod // n_i
        sum_ += a_i * mul_inv(p, n_i) * p
    return sum_ % prod
 
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

In [ ]:
chinese_remainder(buses)

## Day 12
### Part 1

In [ ]:
with open('input12.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]
    directions = [(line[0], int(line[1:])) for line in lines]

In [ ]:
import math

class Ship:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.angle = 0
        
    def forward(self, amount):
        dx = math.cos(math.radians(self.angle))
        dy = math.sin(math.radians(self.angle))
        
        self.x += round(dx*amount)
        self.y += round(dy*amount)
        
    def move(self, cmd, amount):
        if cmd == 'N':
            self.y += amount
        elif cmd == 'W':
            self.x -= amount
        elif cmd == 'S':
            self.y -= amount
        elif cmd == 'E':
            self.x += amount
        elif cmd == 'L':
            self.angle += amount
        elif cmd == 'R':
            self.angle -= amount
        elif cmd == 'F':
            self.forward(amount)
        else:
            print(f"Unknown command {cmd}")
            
    def location(self):
        return self.x, self.y

In [ ]:
ship = Ship()

for direction in directions:
    ship.move(*direction)
sum(abs(l) for l in ship.location())

### Part2

In [ ]:
import math

class ShipV2:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.vx = 10
        self.vy = 1
        
    def rotate(self, angle):
        dx = math.cos(math.radians(angle))
        dy = math.sin(math.radians(angle))
        
        self.vx, self.vy = round((dx * self.vx) - (dy * self.vy)), round((dy * self.vx) + (dx * self.vy))
        
    def forward(self, amount):
        self.x += amount*self.vx
        self.y += amount*self.vy
        
    def move(self, cmd, amount):
        if cmd == 'N':
            self.vy += amount
        elif cmd == 'W':
            self.vx -= amount
        elif cmd == 'S':
            self.vy -= amount
        elif cmd == 'E':
            self.vx += amount
        elif cmd == 'L':
            self.rotate(amount)
        elif cmd == 'R':
            self.rotate(-amount)
        elif cmd == 'F':
            self.forward(amount)
        else:
            print(f"Unknown command {cmd}")
            
    def location(self):
        return self.x, self.y

In [ ]:
ship = ShipV2()

for direction in directions:
    ship.move(*direction)
sum(abs(l) for l in ship.location())

## Day 11
### Part 1

In [ ]:
with open('input11.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
from collections import Counter, defaultdict

def get_empty_area():
    
    area = defaultdict(lambda : defaultdict(lambda :'.'))

    for i, l in enumerate(lines):
        for j, s in enumerate(l):
            area[i][j] = s
            
    return area

def check(area, x, y):
    return Counter(area[i][j] for i in range(x-1, x+2) for j in range(y-1, y+2) if i != x or j != y)

def count(area):
    return Counter(s for l in area.values() for s in l.values())

def iterate(area):
    new_area = get_empty_area()
    for i, l in enumerate(lines):
        for j, s in enumerate(l):
            if s == '.':
                continue
            c = check(area, i, j)

            if c['#'] == 0:
                new_area[i][j] = '#'
            elif c['#'] >= 4:
                new_area[i][j] = 'L'
            else:
                new_area[i][j] = area[i][j]
    return new_area

In [ ]:
area = get_empty_area()

nb_occupied = count(area)['#']

for i in range(1000):
    area = iterate(area)
    new_nb = count(area)['#']
    if new_nb == nb_occupied:
        print(new_nb)
        break
    nb_occupied = new_nb

### Part 2

In [ ]:
def check_line(area, x, y, dx, dy, w, h):
    i, j = x, y
    while True:
        i += dx
        j += dy
        if area[i][j] != '.':
            return area[i][j]
        if i < 0 or i > w or j < 0 or j > h:
            return '.'
        

def check_p2(area, x, y, w, h):
    return Counter([check_line(area, x, y, dx, dy, w, h) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if dx != 0 or dy != 0])


def iterate_p2(area):
    h, w = len(lines), len(lines[0])
    new_area = get_empty_area()
    for i, l in enumerate(lines):
        for j, s in enumerate(l):
            if s == '.':
                continue
            c = check_p2(area, i, j, w, h)

            if c['#'] == 0:
                new_area[i][j] = '#'
            elif c['#'] >= 5:
                new_area[i][j] = 'L'
            else:
                new_area[i][j] = area[i][j]
    return new_area

In [ ]:
area = get_empty_area()

nb_occupied = count(area)['#']

for i in range(1000):
    area = iterate_p2(area)
    new_nb = count(area)['#']
    if new_nb == nb_occupied:
        print(new_nb)
        break
    nb_occupied = new_nb

## Day 10
### Part 1

In [ ]:
with open('input10.txt') as fin:
    lines = [int(line.strip()) for line in fin.readlines()]

In [ ]:
lines.sort()

In [ ]:
from collections import Counter
c = Counter(l2 - l1 for l1, l2 in zip(lines[:-1], lines[1:]))
(c[1]+1)*(c[3]+1)

### Part 2

In [ ]:
c = Counter({0: 1})
for line in lines:
    c[line] = sum(c[e] for e in range(line - 3, line))
print(c.most_common(1)[0][1])

## Day 9
### Part 1

In [ ]:
with open('input09.txt') as fin:
    lines = [int(line.strip()) for line in fin.readlines()]

In [ ]:
def get_sums(numbers):
    return set(nb1 + nb2 for nb1 in set(numbers) for nb2 in set(numbers) if nb1 != nb2)

def check(numbers):
    pre = 25
    for i in range(pre, len(numbers)):
        if numbers[i] not in get_sums(numbers[i-pre:i]):
            return numbers[i]

In [ ]:
check(lines)

### Part 2

In [ ]:
target = check(lines)
max_idx = lines.index(target)

In [ ]:
def find_weakness(numbers):
    for i in range(max_idx):
        for j in range(i, max_idx):
            if sum(numbers[i:j]) == target:
                return min(numbers[i:j]) + max(numbers[i:j])

In [ ]:
find_weakness(lines)

## Day 8
### Part 1

In [ ]:
with open('input08.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
def execute(line):
    cmd, arg = line.split()
    if cmd == 'nop':
        return 1, 0
    arg = int(arg)
    if cmd == 'acc':
        return 1, arg
    if cmd == 'jmp':
        return arg, 0
    
def run(prog):
    idx = 0
    acc = 0
    cache = {0}
    while True:
        off, inc = execute(prog[idx])
        acc += inc
        idx += off
        if idx in cache:
            return False, acc
        if idx >= len(prog):
            return True, acc
        cache.add(idx)
        
_, ans = run(lines)
print(ans)

### Part2

In [ ]:
def fix(prog):
    for i in range(len(prog)):
        if prog[i][:3] == 'acc':
            continue
        fixed = prog.copy()
        if fixed[i][:3] == 'nop':
            fixed[i] = fixed[i].replace('nop', 'jmp')
        else:
            fixed[i] = fixed[i].replace('jmp', 'nop')
        ok, ans = run(fixed)
        if ok:
            print(ans)
            break
            
fix(lines)

## Day 7
### Part 1

In [ ]:
with open('input07.txt') as fin:
    lines = fin.readlines()

In [ ]:
import re
from collections import defaultdict

constraints = {}
reverse = defaultdict(set)

for line in lines:
    bag, content = line.strip().split(' bags contain ')

    constraints[bag] = []

    if not re.match(r"^no other", content):
        content = content.split(", ")
        for item in content:
            match = re.match(r"^(\d) (\w+ \w+) bags?\.?", item)
            if match:
                constraints[bag].append((int(match.group(1)), match.group(2)))
                reverse[match.group(2)].add(bag)

In [ ]:
targets = reverse["shiny gold"].copy()

while True:
    new_targets = set()
    for bag in targets:
        new_targets.update(reverse[bag])
    if new_targets - targets:
        targets.update(new_targets)
    else:
        break
len(targets)

### Part 2

In [ ]:
def recurse(bag):
    if constraints[bag]:
        count = 0
        for qty, subbag in constraints[bag]:
            count += qty*(1+recurse(subbag))
        return count
    else:
        return 0
    
recurse("shiny gold")

## Day 6
### Part 1

In [ ]:
with open('input06.txt') as fin:
    lines = fin.readlines()
    groups = ''.join(lines).split('\n\n')

In [ ]:
def parse_group(group):
    return set("".join(group.split('\n')))

print(sum(len(parse_group(group)) for group in groups))

### Part 2

In [ ]:
def parse_group_p2(group):
    inter_set = parse_group(group)
    for subgroup in group.split('\n'):
        inter_set &= set(subgroup)
    return inter_set

In [ ]:
print(sum(len(parse_group_p2(group)) for group in groups))

## Day 5
### Part 1

In [ ]:
with open('input05.txt') as fin:
    lines = fin.readlines()
    codes = [c.strip() for c in lines]

In [ ]:
import re

def idx(code):
    code = re.sub(r"[FL]", '0', code)
    code = re.sub(r"[BR]", '1', code)
    return int(f"0b{code}", 2)

max(idx(code) for code in codes)

### Part 2

In [ ]:
ids = list(sorted([idx(code) for code in codes]))

for id1, id2 in zip(ids[:-1], ids[1:]):
    if id1 != id2 - 1:
        print(id1 + 1)

## Day 4
### Part 1

In [ ]:
with open('input04.txt') as fin:
    lines = fin.readlines()
    raw_passports = ''.join(lines).split('\n\n')

In [ ]:
import re

def get_passport(raw_line):
    fields = re.split(r"[\s:]", raw_line)
    return {k:v for k,v in zip(fields[:-1:2], fields[1::2])}

In [ ]:
mandatory = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
valid = 0
for raw_pass in raw_passports:
    passport = get_passport(raw_pass)
    if set(passport.keys()) >= mandatory:
        valid += 1
print(valid)

### Part 2

In [ ]:
def vyr(s, low, high):
    r = re.match(r"^\d{4}$", s)
    if r:
        y = int(r.group(0))
        return low <= y <= high
    return False

def vhgt(s):
    r = re.match(r"^(\d+)(cm|in)$", s)
    if r:
        h = int(r.group(1))
        u = r.group(2)
        if u == 'cm':
            return 150 <= h <= 193
        elif u == 'in':
            return 59 <= h <= 76
    return False

rules = {
    'byr': lambda s: vyr(s, 1920, 2002),
    'iyr': lambda s: vyr(s, 2010, 2020),
    'eyr': lambda s: vyr(s, 2020, 2030),
    'hgt': vhgt,
    'hcl': lambda s: bool(re.match(r"^#[0-9a-f]{6}$", s)),
    'ecl': lambda s: bool(re.match(r"^(amb|blu|brn|gry|grn|hzl|oth)$", s)),
    'pid': lambda s: bool(re.match(r"^\d{9}$", s)),
}

In [ ]:
mandatory = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
valid = 0
for raw_pass in raw_passports:
    passport = get_passport(raw_pass)
    if set(passport.keys()) - {'cid'} == set(rules.keys()) and all(rule(passport[key]) for key, rule in rules.items()):
        valid += 1
print(valid)

## Day 3
### Part 1

In [ ]:
with open('input03.txt') as fin:
    lines = fin.readlines()
    trees = [line.strip() for line in lines]
    height, width = len(trees), len(trees[0])

In [ ]:
def step(x, y, dx, dy):
    return x+dx, y+dy

def run(dx=3, dy=1):
    cx, cy = 0, 0
    nb_trees = 0
    while cy < height:
        if trees[cy][cx] == '#':
            nb_trees += 1
        cx, cy = step(cx, cy, dx, dy)
        cx = cx % width
    return nb_trees

In [ ]:
run()

### Part 2

In [ ]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
count = 1
for dx, dy in slopes:
    count *= run(dx, dy)
print(count)

## Day 2
### Part 1

In [ ]:
import re
from collections import Counter

def parse(line):
    low, high, letter, password = re.split(r"[- ]", line.strip())
    
    low, high, letter = int(low), int(high), letter[:-1]
    
    return low, high, letter, password

def valid(line):
    low, high, letter, password = parse(line)
    count = Counter(password)
    
    return count[letter] >= low and count[letter] <= high

with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid(line)])) 

### Part 2

In [ ]:
def valid_p2(line):
    low, high, letter, password = parse(line)
    
    return (password[low - 1] == letter) != (password[high - 1] == letter)

In [ ]:
with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid_p2(line)])) 

## Day 1
### Part 1

In [ ]:
with open('input01.txt') as fin:
    data = [int(d.strip()) for d in fin.readlines()]

In [ ]:
for a in data:
    for b in data:
        if a+b == 2020:
            print(a*b)

### Part 2

In [ ]:
for a in data:
    for b in data:
        for c in data:
            if a+b+c == 2020:
                print(a*b*c)